In [8]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.webdriver import WebDriver
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains


In [25]:


# 필요한 라이브러리 임포트
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from selenium import webdriver
# url
url = 'https://m.place.naver.com/restaurant/1085956231/review/visitor?entry=ple&reviewSort=recent'

# Webdriver headless mode setting
options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size=1920x1080')
options.add_argument("disable-gpu")

# BS4 setting for secondary access
session = requests.Session()
headers = {
    #사용자의 user-agent값
    "User-Agent": "$User value"}

retries = Retry(total=5,
                backoff_factor=0.1,
                status_forcelist=[500, 502, 503, 504])

session.mount('http://', HTTPAdapter(max_retries=retries))


In [32]:
import datetime
from openpyxl import Workbook  # Workbook 클래스를 임포트

# New xlsx file
now = datetime.datetime.now()
xlsx = Workbook()
list_sheet = xlsx.create_sheet('output')
list_sheet.append(['nickname', 'content', 'date', 'revisit'])


In [37]:
from selenium.webdriver.common.by import By
from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter
from openpyxl import Workbook
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
import time
import datetime
import requests
from selenium.webdriver.chrome.service import Service

# url
url = 'https://m.place.naver.com/restaurant/1085956231/review/visitor?entry=ple&reviewSort=recent'

# Webdriver headless mode setting
options = webdriver.ChromeOptions()
# options.add_argument('headless')
options.add_argument('window-size=1920x1080')
options.add_argument("disable-gpu")

# BS4 setting for secondary access
session = requests.Session()
headers = {
    "User-Agent": "user value"}

retries = Retry(total=5,
                backoff_factor=0.1,
                status_forcelist=[500, 502, 503, 504])

session.mount('http://', HTTPAdapter(max_retries=retries))

# New xlsx file
now = datetime.datetime.now()
xlsx = Workbook()
list_sheet = xlsx.create_sheet('output')
list_sheet.append(['nickname', 'content', 'date', 'revisit'])

# Start crawling/scraping!
try:
    # driver = webdriver.Chrome(executable_path=ChromeDriverManager().install(), options=options)
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    res = driver.get(url)
    driver.implicitly_wait(30)

    # Pagedown
    driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)

    try:
        for i in range(30):  # 최대 30번 반복
            driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[2]/div/a').click()
            time.sleep(0.4)
    except Exception as e:
        print('finish')
    else :
        print('30times_finish')

    time.sleep(25)
    html = driver.page_source
    bs = BeautifulSoup(html, 'lxml')
    reviews = bs.select('li.pui__X35jYm.EjjAW')

    for r in reviews:
        # nickname
        nickname = r.select_one('div.pui__JiVbY3 > span.pui__uslU0d')

        # content
        content = r.select_one('div.pui__vn15t2 > a.pui__xtsQN-')

        # date
        date_elements = r.select('div.pui__QKE5Pr > span.pui__gfuUIT > time')
        date = date_elements[0] if date_elements else 'N/A'

        # revisit
        revisit_span = r.select('div.pui__QKE5Pr > span.pui__gfuUIT')
        revisit = revisit_span[1] if len(revisit_span) > 1 else 'N/A'

        # exception handling
        nickname = nickname.text if nickname else ''
        content = content.text if content else ''
        date = date.text if date else ''
        revisit = revisit.text if revisit else ''
        time.sleep(0.06)

        # append sheet
        print(nickname, '/', content, '/', date, '/', revisit)
        list_sheet.append([nickname, content, date, revisit])
        time.sleep(0.06)
    # Save the file
    file_name = 'naver_review_' + now.strftime('%Y-%m-%d_%H-%M-%S') + '.xlsx'
    xlsx.save(file_name)

except Exception as e:
    print(e)
    # Save the file(temp)
    file_name = 'naver_review_' + now.strftime('%Y-%m-%d_%H-%M-%S') + '.xlsx'
    xlsx.save(file_name)


30times_finish
Couldn't find a tree builder with the features you requested: lxml. Do you need to install a parser library?


In [47]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from openpyxl import Workbook
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
import time
import datetime
import pandas as pd
from selenium.webdriver.chrome.service import Service

# CSV 파일에서 사업자명(가게 이름) 리스트 불러오기
csv_file_path = './서울시 영업중 일반음식점.csv'
df = pd.read_csv(csv_file_path, low_memory=False)
store_names = df['사업장명'].tolist()  # '사업장명' 열에서 가게 이름 리스트 추출

# Webdriver 설정
options = webdriver.ChromeOptions()
options.add_argument('window-size=1920x1080')
options.add_argument("disable-gpu")

# 새로운 xlsx 파일 생성
now = datetime.datetime.now()
xlsx = Workbook()
list_sheet = xlsx.create_sheet('output')
list_sheet.append(['nickname', 'content', 'date', 'revisit', 'store_name'])

# 크롤링 시작
try:
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    
    for store_name in store_names:
        # 네이버 일반 검색 페이지로 이동
        search_url = f"https://map.naver.com/v5/search/{store_name}"
        driver.get(search_url)
        driver.implicitly_wait(10)

        try:
            # 첫 번째 검색 결과 클릭
            first_result = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, 'a.place_bluelink'))
            )
            first_result.click()
            time.sleep(3)  # 페이지 로딩 대기

            # 리뷰 탭으로 이동
            try:
                # 리뷰 탭으로 이동하기 전 페이지의 프레임이 변경되었는지 확인하고, 필요 시 프레임 전환
                driver.switch_to.default_content()  # 기본 프레임으로 전환
                review_tab = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, '//a[contains(@href, "visitor?entry=ple&reviewSort=recent")]'))
                )
                review_tab.click()
                time.sleep(3)  # 리뷰 페이지 로딩 대기
            except (TimeoutException, NoSuchElementException) as e:
                print(f"리뷰 탭을 찾을 수 없음: {store_name}, 에러: {e}")
                continue

            # 페이지 로딩 기다림
            time.sleep(3)   

            # Pagedown
            driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
            time.sleep(2)

            try:
                for i in range(30):  # 최대 30번 반복
                    # "더보기" 버튼 클릭 (수정된 부분)
                    driver.find_element(By.XPATH, '//a[contains(@class, "more_btn")]').click()
                    time.sleep(0.4)
            except Exception as e:
                print('finish')
            else:
                print('30times_finish')

            time.sleep(25)
            html = driver.page_source
            bs = BeautifulSoup(html, 'lxml')
            reviews = bs.select('li.pui__X35jYm.EjjAW')

            for r in reviews:
                # nickname
                nickname = r.select_one('div.pui__JiVbY3 > span.pui__uslU0d')

                # content
                content = r.select_one('div.pui__vn15t2 > a.pui__xtsQN-')

                # date
                date_elements = r.select('div.pui__QKE5Pr > span.pui__gfuUIT > time')
                date = date_elements[0] if date_elements else 'N/A'

                # revisit
                revisit_span = r.select('div.pui__QKE5Pr > span.pui__gfuUIT')
                revisit = revisit_span[1] if len(revisit_span) > 1 else 'N/A'

                # 예외 처리
                nickname = nickname.text if nickname else ''
                content = content.text if content else ''
                date = date.text if date else ''
                revisit = revisit.text if revisit else ''
                time.sleep(0.06)

                # 시트에 추가 (store_name도 함께 기록)
                print(nickname, '/', content, '/', date, '/', revisit)
                list_sheet.append([nickname, content, date, revisit, store_name])
                time.sleep(0.06)

        except (TimeoutException, NoSuchElementException) as e:
            print(f"검색 결과를 찾을 수 없음: {store_name}")
            continue

    # 파일 저장
    file_name = 'naver_review_' + now.strftime('%Y-%m-%d_%H-%M-%S') + '.xlsx'
    xlsx.save(file_name)

except Exception as e:
    print(e)
    # 임시 파일 저장
    file_name = 'naver_review_' + now.strftime('%Y-%m-%d_%H-%M-%S') + '.xlsx'
    xlsx.save(file_name)
finally:
    driver.quit()


검색 결과를 찾을 수 없음: 매취랑 동대문점


KeyboardInterrupt: 

In [3]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from openpyxl import Workbook
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import time
import datetime
import pandas as pd

# CSV 파일에서 사업자명(가게 이름) 리스트 불러오기
csv_file_path = './서울시 영업중 일반음식점.csv'
df = pd.read_csv(csv_file_path, low_memory=False)
store_names = df['사업장명'].tolist()  # '사업장명' 열에서 가게 이름 리스트 추출

# Webdriver 설정
options = Options()
options.add_argument('window-size=1920x1080')
options.add_argument("disable-gpu")

# 새로운 xlsx 파일 생성
now = datetime.datetime.now()
xlsx = Workbook()
list_sheet = xlsx.create_sheet('output')
list_sheet.append(['nickname', 'content', 'date', 'revisit', 'store_name'])

# 크롤링 시작
try:
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    
    for store_name in store_names:
        # 네이버 일반 검색 페이지로 이동
        search_url = f"https://map.naver.com/v5/search/{store_name}"
        driver.get(search_url)
        driver.implicitly_wait(10)

        try:
            # 첫 번째 검색 결과 클릭
            first_result = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, 'a.place_bluelink'))
            )
            first_result.click()
            time.sleep(3)  # 페이지 로딩 대기

            # 리뷰 탭으로 이동
            try:
                # 페이지의 프레임이 변경되었는지 확인하고, 필요 시 프레임 전환
                driver.switch_to.default_content()  # 기본 프레임으로 전환

                # '리뷰' 탭을 클릭 (수정된 XPATH)
                review_tab = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, '//span[text()="리뷰"]'))
                )
                review_tab.click()
                time.sleep(3)  # 리뷰 페이지 로딩 대기
            except (TimeoutException, NoSuchElementException) as e:
                print(f"리뷰 탭을 찾을 수 없음: {store_name}, 에러: {e}")
                continue

            # 페이지 로딩 기다림
            time.sleep(3)

            # Pagedown
            driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
            time.sleep(2)

            try:
                for i in range(30):  # 최대 30번 반복
                    # "더보기" 버튼 클릭 (수정된 부분)
                    driver.find_element(By.XPATH, '//a[contains(@class, "more_btn")]').click()
                    time.sleep(0.4)
            except Exception as e:
                print('finish')
            else:
                print('30times_finish')

            time.sleep(25)
            html = driver.page_source
            bs = BeautifulSoup(html, 'lxml')
            reviews = bs.select('li.pui__X35jYm.EjjAW')

            for r in reviews:
                # nickname
                nickname = r.select_one('div.pui__JiVbY3 > span.pui__uslU0d')

                # content
                content = r.select_one('div.pui__vn15t2 > a.pui__xtsQN-')

                # date
                date_elements = r.select('div.pui__QKE5Pr > span.pui__gfuUIT > time')
                date = date_elements[0] if date_elements else 'N/A'

                # revisit
                revisit_span = r.select('div.pui__QKE5Pr > span.pui__gfuUIT')
                revisit = revisit_span[1] if len(revisit_span) > 1 else 'N/A'

                # 예외 처리
                nickname = nickname.text if nickname else ''
                content = content.text if content else ''
                date = date.text if date else ''
                revisit = revisit.text if revisit else ''
                time.sleep(0.06)

                # 시트에 추가 (store_name도 함께 기록)
                print(nickname, '/', content, '/', date, '/', revisit)
                list_sheet.append([nickname, content, date, revisit, store_name])
                time.sleep(0.06)

        except (TimeoutException, NoSuchElementException) as e:
            print(f"검색 결과를 찾을 수 없음: {store_name}, 에러: {e}")
            continue

    # 파일 저장
    file_name = 'naver_review_' + now.strftime('%Y-%m-%d_%H-%M-%S') + '.xlsx'
    xlsx.save(file_name)

except Exception as e:
    print(e)
    # 임시 파일 저장
    file_name = 'naver_review_' + now.strftime('%Y-%m-%d_%H-%M-%S') + '.xlsx'
    xlsx.save(file_name)
finally:
    driver.quit()


검색 결과를 찾을 수 없음: 매취랑 동대문점, 에러: Message: 
Stacktrace:
	GetHandleVerifier [0x009C8923+23283]
	(No symbol) [0x0098E934]
	(No symbol) [0x008C0733]
	(No symbol) [0x0090326F]
	(No symbol) [0x009034AB]
	(No symbol) [0x0093EE42]
	(No symbol) [0x00924464]
	(No symbol) [0x0093CB8D]
	(No symbol) [0x009241B6]
	(No symbol) [0x008F8017]
	(No symbol) [0x008F890D]
	GetHandleVerifier [0x00ABA5F3+1013699]
	GetHandleVerifier [0x00AC3E4C+1052700]
	GetHandleVerifier [0x00ABD4B4+1025668]
	GetHandleVerifier [0x009EEA2B+179195]
	(No symbol) [0x00996833]
	(No symbol) [0x00993198]
	(No symbol) [0x00993337]
	(No symbol) [0x0098B4BE]
	BaseThreadInitThunk [0x7620FCC9+25]
	RtlGetAppContainerNamedObjectPath [0x778E80CE+286]
	RtlGetAppContainerNamedObjectPath [0x778E809E+238]

검색 결과를 찾을 수 없음: 치카바, 에러: Message: 
Stacktrace:
	GetHandleVerifier [0x009C8923+23283]
	(No symbol) [0x0098E934]
	(No symbol) [0x008C0733]
	(No symbol) [0x0090326F]
	(No symbol) [0x009034AB]
	(No symbol) [0x0093EE42]
	(No symbol) [0x00924464]
	(No

KeyboardInterrupt: 

In [4]:
##의룡님 코드

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from openpyxl import Workbook
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
import time
import datetime
import pandas as pd
from selenium.webdriver.chrome.service import Service

# CSV 파일에서 사업자명(가게 이름) 리스트 불러오기
csv_file_path = './서울시 영업중 일반음식점.csv'
df = pd.read_csv(csv_file_path, low_memory=False)
#store_names = df['사업장명'].tolist()  # '사업장명' 열에서 가게 이름 리스트 추출
store_names =['잔치루']
# Webdriver 설정
options = webdriver.ChromeOptions()
options.add_argument('window-size=1920x1080')
options.add_argument("disable-gpu")

# 새로운 xlsx 파일 생성
now = datetime.datetime.now()
xlsx = Workbook()
list_sheet = xlsx.create_sheet('output')
list_sheet.append(['nickname', 'content', 'date', 'revisit', 'store_name'])

# 크롤링 시작
try:
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    
    for store_name in store_names:
        # 네이버 일반 검색 페이지로 이동
        search_url = f"https://search.naver.com/search.naver?query={store_name}"
        driver.get(search_url)
        driver.implicitly_wait(10)

        try:
            # 첫 번째 검색 결과 클릭 (네이버 지도나 가게 페이지로 이동)
            first_result = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, 'a.place_bluelink'))
            )
            first_result.click()
            time.sleep(3)

            # 리뷰 탭으로 이동 (가게 페이지에서 리뷰로 이동)
            try:
                # 프레임이 아닌 기본 페이지에서 XPATH로 리뷰 탭을 클릭
                review_tab = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, '//a[contains(@href, "visitor?entry=ple&reviewSort=recent")]'))
                )
                review_tab.click()
                time.sleep(3)
            except (TimeoutException, NoSuchElementException):
                print(f"리뷰 탭을 찾을 수 없음: {store_name}")
                continue

            # 페이지 로딩 기다림
            time.sleep(3)

            # Pagedown
            driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
            time.sleep(2)

            try:
                for i in range(30):  # 최대 30번 반복
                    driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[2]/div/a').click()
                    time.sleep(0.4)
            except Exception as e:
                print('finish')
            else:
                print('30times_finish')

            time.sleep(25)
            html = driver.page_source
            bs = BeautifulSoup(html, 'lxml')
            reviews = bs.select('li.pui__X35jYm.EjjAW')

            for r in reviews:
                # nickname
                nickname = r.select_one('div.pui__JiVbY3 > span.pui__uslU0d')

                # content
                content = r.select_one('div.pui__vn15t2 > a.pui__xtsQN-')

                # date
                date_elements = r.select('div.pui__QKE5Pr > span.pui__gfuUIT > time')
                date = date_elements[0] if date_elements else 'N/A'

                # revisit
                revisit_span = r.select('div.pui__QKE5Pr > span.pui__gfuUIT')
                revisit = revisit_span[1] if len(revisit_span) > 1 else 'N/A'

                # 예외 처리
                nickname = nickname.text if nickname else ''
                content = content.text if content else ''
                date = date.text if date else ''
                revisit = revisit.text if revisit else ''
                time.sleep(0.06)

                # 시트에 추가 (store_name도 함께 기록)
                print(nickname, '/', content, '/', date, '/', revisit)
                list_sheet.append([nickname, content, date, revisit, store_name])
                time.sleep(0.06)

        except (TimeoutException, NoSuchElementException) as e:
            print(f"검색 결과를 찾을 수 없음: {store_name}")
            continue

    # 파일 저장
    file_name = 'naver_review_' + now.strftime('%Y-%m-%d_%H-%M-%S') + '.xlsx'
    xlsx.save(file_name)

except Exception as e:
    print(e)
    # 임시 파일 저장
    file_name = 'naver_review_' + now.strftime('%Y-%m-%d_%H-%M-%S') + '.xlsx'
    xlsx.save(file_name)
finally:
    driver.quit()


리뷰 탭을 찾을 수 없음: 잔치루


In [7]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from openpyxl import Workbook
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import time
import datetime
import pandas as pd

# CSV 파일에서 사업자명(가게 이름) 리스트 불러오기
csv_file_path = './서울시 영업중 일반음식점.csv'
df = pd.read_csv(csv_file_path, low_memory=False)
store_names = df['사업장명'].tolist()  # '사업장명' 열에서 가게 이름 리스트 추출

# Webdriver 설정
options = Options()
options.add_argument('window-size=1920x1080')
options.add_argument("disable-gpu")

# 새로운 xlsx 파일 생성
now = datetime.datetime.now()
xlsx = Workbook()
list_sheet = xlsx.create_sheet('output')
list_sheet.append(['nickname', 'content', 'date', 'revisit', 'store_name'])

# 크롤링 시작
try:
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    
    for store_name in store_names:
        # 네이버 지도 검색 페이지로 이동
        search_url = f"https://map.naver.com/v5/search/{store_name}"
        driver.get(search_url)
        driver.implicitly_wait(10)

        try:
            # 첫 번째 검색 결과 클릭
            first_result = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, 'a.place_bluelink'))
            )
            first_result.click()
            time.sleep(3)  # 페이지 로딩 대기

            # 리뷰 탭으로 이동
            try:
                # 프레임 확인 및 전환
                WebDriverWait(driver, 10).until(
                    EC.frame_to_be_available_and_switch_to_it((By.CSS_SELECTOR, "iframe#entryIframe"))
                )
                print("프레임 전환 성공")

                # 다양한 방법으로 리뷰탭 클릭 시도
                try:
                    review_tab = WebDriverWait(driver, 5).until(
                        EC.element_to_be_clickable((By.XPATH, '//a[contains(@href, "visitor?entry=ple&reviewSort=recent")]'))
                    )
                    review_tab.click()
                except (TimeoutException, NoSuchElementException):
                    print("첫 번째 리뷰탭 탐색 실패, span 기반 탐색 시도")

                    # '리뷰'가 span 태그일 경우 탐색
                    try:
                        review_tab = WebDriverWait(driver, 5).until(
                            EC.element_to_be_clickable((By.XPATH, '//span[text()="리뷰"]'))
                        )
                        review_tab.click()
                    except (TimeoutException, NoSuchElementException):
                        print(f"리뷰 탭을 찾을 수 없음: {store_name}")
                        continue

                time.sleep(3)  # 리뷰 페이지 로딩 대기
                print(f"{store_name}: 리뷰탭 클릭 성공")

            except (TimeoutException, NoSuchElementException) as e:
                print(f"리뷰 탭을 찾을 수 없음: {store_name}, 에러: {e}")
                continue

            # 페이지 로딩 기다림
            time.sleep(3)

            # Pagedown
            driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
            time.sleep(2)

            try:
                for i in range(30):  # 최대 30번 반복
                    # "더보기" 버튼 클릭
                    driver.find_element(By.XPATH, '//a[contains(@class, "more_btn")]').click()
                    time.sleep(0.4)
            except Exception as e:
                print('finish')
            else:
                print('30times_finish')

            time.sleep(25)
            html = driver.page_source
            bs = BeautifulSoup(html, 'lxml')
            reviews = bs.select('li.pui__X35jYm.EjjAW')

            for r in reviews:
                # nickname
                nickname = r.select_one('div.pui__JiVbY3 > span.pui__uslU0d')

                # content
                content = r.select_one('div.pui__vn15t2 > a.pui__xtsQN-')

                # date
                date_elements = r.select('div.pui__QKE5Pr > span.pui__gfuUIT > time')
                date = date_elements[0] if date_elements else 'N/A'

                # revisit
                revisit_span = r.select('div.pui__QKE5Pr > span.pui__gfuUIT')
                revisit = revisit_span[1] if len(revisit_span) > 1 else 'N/A'

                # 예외 처리
                nickname = nickname.text if nickname else ''
                content = content.text if content else ''
                date = date.text if date else ''
                revisit = revisit.text if revisit else ''
                time.sleep(0.06)

                # 시트에 추가 (store_name도 함께 기록)
                print(nickname, '/', content, '/', date, '/', revisit)
                list_sheet.append([nickname, content, date, revisit, store_name])
                time.sleep(0.06)

        except (TimeoutException, NoSuchElementException) as e:
            print(f"검색 결과를 찾을 수 없음: {store_name}, 에러: {e}")
            continue

    # 파일 저장
    file_name = 'naver_review_' + now.strftime('%Y-%m-%d_%H-%M-%S') + '.xlsx'
    xlsx.save(file_name)

except Exception as e:
    print(e)
    # 임시 파일 저장
    file_name = 'naver_review_' + now.strftime('%Y-%m-%d_%H-%M-%S') + '.xlsx'
    xlsx.save(file_name)
finally:
    driver.quit()


Message: disconnected: unable to send message to renderer
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=127.0.6533.101)
Stacktrace:
	GetHandleVerifier [0x009C8923+23283]
	(No symbol) [0x0098E934]
	(No symbol) [0x008C0733]
	(No symbol) [0x008ABD9C]
	(No symbol) [0x008ABD26]
	(No symbol) [0x008AAB0B]
	(No symbol) [0x008AB31D]
	(No symbol) [0x008C429F]
	(No symbol) [0x008C4A32]
	(No symbol) [0x008C7A88]
	(No symbol) [0x008C7B00]
	(No symbol) [0x00903027]
	(No symbol) [0x009034AB]
	(No symbol) [0x0093EE42]
	(No symbol) [0x00924464]
	(No symbol) [0x0093CB8D]
	(No symbol) [0x009241B6]
	(No symbol) [0x008F8017]
	(No symbol) [0x008F890D]
	GetHandleVerifier [0x00ABA5F3+1013699]
	GetHandleVerifier [0x00AC3E4C+1052700]
	GetHandleVerifier [0x00ABD4B4+1025668]
	GetHandleVerifier [0x009EEA2B+179195]
	(No symbol) [0x00996833]
	(No symbol) [0x00993198]
	(No symbol) [0x00993337]
	(No symbol) [0x0098B4BE]
	BaseThreadInitThunk [0x7620FCC9+25]
	R

In [11]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from openpyxl import Workbook
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import time
import datetime
import pandas as pd

# CSV 파일에서 사업자명(가게 이름) 리스트 불러오기
csv_file_path = './서울시 영업중 일반음식점.csv'
df = pd.read_csv(csv_file_path, low_memory=False)
store_names = df['사업장명'].tolist()  # '사업장명' 열에서 가게 이름 리스트 추출

# Webdriver 설정
options = Options()
options.add_argument('window-size=1920x1080')
options.add_argument("disable-gpu")

# 새로운 xlsx 파일 생성
now = datetime.datetime.now()
xlsx = Workbook()
list_sheet = xlsx.create_sheet('output')
list_sheet.append(['nickname', 'content', 'date', 'revisit', 'store_name'])


In [18]:

# # 크롤링 시작
# try:
#     driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    
#     for store_name in store_names:
#         try:
#             # 네이버 지도 검색 페이지로 이동
#             search_url = f"https://map.naver.com/v5/search/{store_name}"
#             driver.get(search_url)
#             driver.implicitly_wait(10)
#             # 프레임 전환 시도
#             WebDriverWait(driver, 10).until(
#                 EC.frame_to_be_available_and_switch_to_it((By.CSS_SELECTOR, "iframe#entryIframe"))
#             )
#             print("프레임 전환 성공")
#             # 첫 번째 검색 결과 클릭
#             first_result = WebDriverWait(driver, 20).until(
#                 EC.element_to_be_clickable((By.CSS_SELECTOR, 'a.place_bluelink'))
#             )
#             first_result.click()
#             print(f"'{store_name}'의 첫 번째 검색 결과 클릭 성공")
#             time.sleep(3)  # 페이지 로딩 대기

            

#             # 리뷰 탭 클릭 시도
#             try:
#                 review_tab = WebDriverWait(driver, 10).until(
#                     EC.element_to_be_clickable((By.XPATH, '//a[contains(@href, "visitor?entry=ple&reviewSort=recent")]'))
#                 )
#                 review_tab.click()
#                 print(f"'{store_name}'의 리뷰탭 클릭 성공")
#                 time.sleep(3)  # 리뷰 페이지 로딩 대기
#             except TimeoutException:
#                 # '리뷰' 텍스트로 찾기 시도
#                 review_tab = WebDriverWait(driver, 10).until(
#                     EC.element_to_be_clickable((By.XPATH, '//span[text()="리뷰"]'))
#                 )
#                 review_tab.click()
#                 print(f"'{store_name}'의 '리뷰' 탭 클릭 성공 (대체 방법)")
#                 time.sleep(3)
            
#             # 리뷰를 찾는 추가 코드 ...
#             time.sleep(3)  # 페이지 로딩 대기

#         except (TimeoutException, NoSuchElementException) as e:
#             print(f"검색 결과를 찾을 수 없음: {store_name}, 에러: {e}")
#             continue

try:
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    
    for store_name in store_names:
        try:
            # 네이버 지도 검색 페이지로 이동
            search_url = f"https://map.naver.com/v5/search/{store_name}"
            driver.get(search_url)
            driver.implicitly_wait(10)

            # 첫 번째 검색 결과 클릭 (프레임 전환 없이)
            try:
                first_result = WebDriverWait(driver, 20).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, 'a.place_bluelink'))
                )
                first_result.click()
                print(f"'{store_name}'의 첫 번째 검색 결과 클릭 성공")
            except TimeoutException:
                print(f"검색 결과를 찾을 수 없음: {store_name}, 첫 번째 항목 클릭 실패")
                continue
            
            time.sleep(3)  # 페이지 로딩 대기

            # 첫 번째 항목 클릭 후 프레임 전환 시도
            try:
                WebDriverWait(driver, 10).until(
                    EC.frame_to_be_available_and_switch_to_it((By.CSS_SELECTOR, "iframe#entryIframe"))
                )
                print("첫 번째 항목의 프레임 전환 성공")
            except TimeoutException:
                print(f"프레임 전환 실패: {store_name}")
                continue

            time.sleep(3)  # 프레임 로딩 대기

            # 리뷰 탭 클릭 시도 (여러 가지 방법으로 시도)
            try:
                # 방법 1: 리뷰 탭을 찾는 첫 번째 시도 (XPath 사용)
                review_tab = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, '//a[contains(@href, "review") and contains(@role, "tab")]'))
                )
                review_tab.click()
                print(f"'{store_name}'의 리뷰탭 클릭 성공 (방법 1 - XPath)")
            except TimeoutException:
                try:
                    # 방법 2: '리뷰' 텍스트로 찾기 (XPath로 '리뷰' 텍스트가 포함된 요소 클릭)
                    review_tab = WebDriverWait(driver, 10).until(
                        EC.element_to_be_clickable((By.XPATH, '//span[contains(text(), "리뷰")]'))
                    )
                    review_tab.click()
                    print(f"'{store_name}'의 리뷰탭 클릭 성공 (방법 2 - '리뷰' 텍스트)")
                except TimeoutException:
                    try:
                        # 방법 3: CSS 선택자로 찾기
                        review_tab = WebDriverWait(driver, 10).until(
                            EC.element_to_be_clickable((By.CSS_SELECTOR, 'a[href*="review"] span.veBoZ'))
                        )
                        review_tab.click()
                        print(f"'{store_name}'의 리뷰탭 클릭 성공 (방법 3 - CSS 선택자)")
                    except TimeoutException:
                        print(f"'{store_name}'의 리뷰탭 클릭 실패 (모든 방법 실패)")
                        continue

 
            time.sleep(3)  # 리뷰 페이지 로딩 대기

            # 리뷰 크롤링 코드 추가
            reviews = driver.find_elements(By.CSS_SELECTOR, 'div.review_text')
            for review in reviews:
                print(review.text)  # 또는 엑셀에 저장

            # 프레임 전환 원복
            driver.switch_to.default_content()

        except (TimeoutException, NoSuchElementException) as e:
            print(f"검색 및 리뷰탭 접근 실패: {store_name}, 에러: {e}")
            continue

finally:
    driver.quit()


#         # 파일 저장
#         file_name = 'naver_review_' + now.strftime('%Y-%m-%d_%H-%M-%S') + '.xlsx'
#         xlsx.save(file_name)

# except Exception as e:
#     print(f"크롤링 중 에러 발생: {e}")
#     file_name = 'naver_review_' + now.strftime('%Y-%m-%d_%H-%M-%S') + '.xlsx'
#     xlsx.save(file_name)

# finally:
#     driver.quit()



검색 결과를 찾을 수 없음: 매취랑 동대문점, 첫 번째 항목 클릭 실패
검색 결과를 찾을 수 없음: 치카바, 첫 번째 항목 클릭 실패
검색 결과를 찾을 수 없음: 오고보, 첫 번째 항목 클릭 실패
검색 결과를 찾을 수 없음: 또봉이통닭 가락본동점, 첫 번째 항목 클릭 실패


KeyboardInterrupt: 

In [23]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from openpyxl import Workbook
import time

# 크롬 옵션 설정
options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('window-size=1920x1080')
options.add_argument("disable-gpu")

# 크롬 드라이버 초기화
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# 엑셀 파일 생성
xlsx = Workbook()
sheet = xlsx.active
sheet.title = 'reviews'
sheet.append(['store_name', 'review_url', 'nickname', 'content', 'date', 'revisit'])

try:
    # 샘플 가게 이름들
    store_names = ''  # 실제 CSV 파일 읽기 기능을 추가해도 좋습니다.

    for store_name in store_names:
        # 네이버 지도에서 가게 검색
        search_url = f"https://map.naver.com/v5/search/{store_name}"
        driver.get(search_url)
        time.sleep(3)  # 페이지 로딩 대기
        
        try:
            # 첫 번째 가게 클릭
            first_result = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, 'a.place_bluelink'))
            )
            first_result.click()
            time.sleep(3)  # 페이지 로딩 대기
            
            # iframe 전환 (iframe이 로드될 때까지 대기)
            WebDriverWait(driver, 10).until(
                EC.frame_to_be_available_and_switch_to_it((By.CSS_SELECTOR, 'iframe#entryIframe'))
            )
            print("프레임 전환 성공")
            
            # 리뷰 탭 URL 추출
            review_tab = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, '//a[contains(@href, "visitor?entry=ple&reviewSort=recent")]'))
            )
            review_url = review_tab.get_attribute("href")
            print(f"'{store_name}'의 리뷰 URL: {review_url}")
            
            # 리뷰 페이지로 이동
            driver.get(review_url)
            time.sleep(3)
            
            # 리뷰 페이지에서 리뷰 크롤링 시작
            reviews = driver.find_elements(By.CSS_SELECTOR, 'li.pui__X35jYm.EjjAW')
            for review in reviews:
                nickname = review.find_element(By.CSS_SELECTOR, 'div.pui__JiVbY3 > span.pui__uslU0d').text
                content = review.find_element(By.CSS_SELECTOR, 'div.pui__vn15t2 > a.pui__xtsQN-').text
                date = review.find_element(By.CSS_SELECTOR, 'div.pui__QKE5Pr > span.pui__gfuUIT > time').text
                
                print(f"리뷰 - 닉네임: {nickname}, 내용: {content}, 날짜: {date}")
                sheet.append([store_name, review_url, nickname, content, date, ''])

            time.sleep(3)  # 다음 가게로 이동하기 전 대기

        except TimeoutException:
            print(f"검색 결과를 찾을 수 없음: {store_name}")
            continue

    # 엑셀 파일 저장
    xlsx.save('naver_reviews.xlsx')

finally:
    driver.quit()


검색 결과를 찾을 수 없음: 강남 맛집
검색 결과를 찾을 수 없음: 압구정 맛집
검색 결과를 찾을 수 없음: 홍대 맛집
